<a href="https://colab.research.google.com/github/G3niy/MS/blob/main/SMO_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

import random
import pandas as pd
import matplotlib.pyplot as plt
from statistics import mean

client_num = 120 
entry_low = 1
entry_high = 9
entry_mode = 5
service_low = 1
service_high = 9
service_mode = 5
arr_entry = []
arr_service = []
time_list = []
queue_list = []
service_list = []
service_time_list = []
service_client_list = []
client_stek = []
service_bool  = True
service_time = 0
all_time = 1
service_transact = 0
next_transact = 1
deliver = []
for i in range(client_num):
    arr_entry.append(round(random.triangular(entry_low, entry_high, entry_mode)))
    arr_service.append(round(random.triangular(service_low, service_high, service_mode)))
timiAll = 150
arrival_time = []
arrival_time.append(0)
for i in range(client_num):
    arrival_time.append(arrival_time[-1] + arr_entry[i])
time_list.append(0)
queue_list.append(0)
service_list.append(service_bool)
service_time_list.append(service_time)
service_client_list.append(0)
while (all_time < timiAll):
    if(service_bool): 
        service_time += 1
    if(service_time == arr_service[service_transact]):
        service_bool = False
        service_time = 0
        deliver.append(service_transact)
        service_transact = -1       
    if(all_time == arrival_time[next_transact]):
        client_stek.append(next_transact)
        if (next_transact < client_num - 1):
            next_transact += 1       
    if(not service_bool) and (len(client_stek) != 0):
        service_transact = client_stek[0]
        client_stek.pop(0)
        service_bool = True
        service_time = 0
    time_list.append(all_time)
    queue_list.append(len(client_stek))
    service_list.append(service_bool)
    service_time_list.append(service_time)
    service_client_list.append(service_transact)
    all_time += 1
print('Было полностью обслужено транзактов:', max(deliver)+1, 'Средняя длина очереди:', '%.1f' % mean(queue_list), 'Max queue lenght: ', max(queue_list))
pd.DataFrame({ 'Общее время': time_list, 'Длина очереди': queue_list, 'Состояние УО': service_list, 'Обслуживаемый транзакт': service_client_list, 'Время занятия УО': service_time_list })

Было полностью обслужено транзактов: 26 Средняя длина очереди: 1.3 Max queue lenght:  4


,Общее время,Длина очереди,Состояние УО,Обслуживаемый транзакт,Время занятия УО
0,0,0,True,0,0
1,1,0,True,0,1
2,2,0,True,0,2
3,3,0,True,0,3
4,4,1,True,0,4
...,...,...,...,...,...
145,145,3,True,26,2
146,146,3,True,26,3
147,147,4,True,26,4
148,148,4,True,26,5


## 4) Событийная модель

In [ ]:
#@title Время работы
max_global_time = 220 #@param {type:"slider", min:20, max:720, step:10}

In [ ]:
#@title Предварительные расчеты
arrival_time = []
arrival_time.append(0)
for i in range(max_transact_num):
    arrival_time.append(arrival_time[-1] + arr_entry[i])
arrival_time

In [ ]:
#@title Код
queue = []
UO  = True
trans_in = 0
trans_s = -1
s_free_time = 0 + arr_service[0]
curr_time = 0
ev = 0

n_event = []
event = []
queue_len = []
curr_trans_s = []
time = []
queue.append(0)
trans_in += 1

event.append(0)
time.append(0)
queue_len.append(len(queue))
curr_trans_s.append(trans_s)
n_event.append('input')


while(True):
    event_time = min(arrival_time[trans_in], s_free_time, max_global_time)
    curr_time = event_time
    if curr_time >= max_global_time:
        event_name = 'end'
        break
    if curr_time == arrival_time[trans_in]:
        queue.append(trans_in)
        trans_in += 1
        event_name = 'input'
    else:
        UO = False
        event_name = 'complete'
    if (len(queue) != 0) and not UO:
        trans_s = queue[0]
        queue.pop(0)
        UO  = True
        s_free_time = curr_time + arr_service[trans_s]
    elif(len(queue) == 0):
        s_free_time = 1000
        trans_s = -1

    ev += 1
    event.append(ev)
    n_event.append(event_name)
    queue_len.append(len(queue))
    curr_trans_s.append(trans_s)
    time.append(curr_time)


In [ ]:
#@title Вывод
check_event = pd.DataFrame({
    '# события': event,
    'Время': time,
    'Событие': n_event,
    'Длина очереди': queue_len,
    'Обслуживаемый транзакт': curr_trans_s
    })
check_event

,# события,Время,Событие,Длина очереди,Обслуживаемый транзакт
0,0,0,input,1,-1
1,1,4,complete,0,0
2,2,6,input,1,0
3,3,8,complete,0,1
4,4,14,input,1,1
...,...,...,...,...,...
64,64,195,complete,1,31
65,65,199,complete,0,32
66,66,201,input,1,32
67,67,204,input,2,32
